In [1]:
import pandas as pd
import os, re, shutil
import csv 
import xlrd
import math
import time
import numpy as np
import logging
import mhi.pscad
import networkx as nx
import matplotlib.pyplot as plt
import datetime
#import numpy as np
import sys, os
sys.path.insert(0, '..')
import os.path

In [2]:
from usrlibs.fileUtils import UsrFileUtils as utils
from usrlibs.UsrFuncDefP2P import UsrFuncDefs
P2P = UsrFuncDefs()
from usrlibs.SystemStudy import UsrSystemStudy
ss = UsrSystemStudy()
from usrlibs.emailUtils import UsrMailUtils
mailutils = UsrMailUtils()


In [3]:
time_start = time.time()

compiler_list = {
    'GFortran 4.6.2': '.gf46',
    'Intel(R) Visual Fortran Compiler 19.0.1.144': '.if15_x86',
    'Intel(R) Visual Fortran Compiler 19.0.1.144': '.if15_x86'}

fortran_version = sorted(compiler_list.keys())[1] #0:Gfortran, 1: Intel-86, 2: Intel-64bit
fortran_ext = compiler_list[fortran_version]

# 1. Read Data and Proceed

In [4]:
Freq = 60

In [5]:
working_dir = os.getcwd()
#retrieves the current working directory and assigns it to the variable 'working_dir'.
#The os.getcwd() function is used to get the current working directory where your Python script is located
project_infor_dir = working_dir + "\\" + "Project_Info"
#This line constructs a path to a directory named "Project_Info" by concatenating the 'working_dir' with the subdirectory name
raw_files = utils.get_all_file_names(project_infor_dir, ".xlsm")
file_name = project_infor_dir + "\\" + raw_files[0] + '.xlsm'
#constructs a specific file name by concatenating the 'project_infor_dir', a backslash (\), the third element (index 2) from the raw_files list, and the ".xlsm" extension

#wb = xlrd.open_workbook(file_name)

In [6]:
print(file_name)

d:\Work_Coding\PSCAD\Project_Info\River_Start_Solar_IV.xlsm


In [7]:
# read excel file
xldataframe = pd.ExcelFile(file_name)
sheets = xldataframe.sheet_names
df = xldataframe.parse(sheets[3])
#why 'df' also contains sheets[1]

c:\Users\Nguyen Tien Bac\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [8]:
#print(df)

In [9]:
#print(sheets)

## Read data equivalent PI section

In [10]:
#This code only read sheets[0,2,3]
# Define PI section
PILineDef = {}
PIattr = {}
PIpara = {}
# Define 3-winding MPTs


In [11]:
BigOrder = []  #ultimate list of list of all feeders in the plant
order = []     #working list that the buses of the current feeder is saved in until it is appended to the BigOrder feeder list
allFdrLen = []
fdrLen = []
i = 0
PIdata = []
cnt = 0
Runs = []
while float(df.iloc[i,1]) > 0:
    pair = [int(df.iloc[i,1]), int(df.iloc[i,2])]
    order.append(pair)
    line = [df.iloc[i,x] for x in [1, 2, 4, 23, 24, 27, 25, 26, 27, 5]] # last entry is for the cable length
    fdrLen.append([int(df.iloc[i,1]), int(df.iloc[i,2]), int(df.iloc[i, 5])])
    #PIdata.append(PIdata.copy()) 
    #PIdata[cnt] = P2P.usr_LineData(line, PIdata[cnt])
    PItemp = P2P.usr_LineData(line, PIattr.copy(), PIpara.copy()) 
    # .copy() to avoid mutual editable of the dictionaries
    #it copy data from usr_LineData dictionary in UsrFuncDefP2P file with each i value 
    PIdata.append(PItemp)
#    print(pair) 
    cnt += 1
    if (int(df.iloc[i,2]) > 100000) & (int(df.iloc[i,2]) % 10000 == 0):
        #why should be greater than 1000000 and 10000
        BigOrder.append(order)
        #contains 'from' node and 'to' node addresses
        order = []

        allFdrLen.append(fdrLen)
        #allFdrLen contains data in '4 UG collection sys impedance' like 'from','to','length'
        fdrLen = []
    i += 1
n_rows = i
#for p in PIdata:
#    print(p[0]['LConn'], p[0]['RConn'])
#for (i, o) in enumerate(BigOrder):
#    print(i)
#    print(o)

In [12]:
allFdrLen

[[[10006, 10007, 1198],
  [10007, 11000, 902],
  [10008, 11000, 1593],
  [11000, 10009, 3922],
  [10009, 10012, 3481],
  [10012, 10010, 1343],
  [10010, 12000, 1477],
  [10005, 12000, 5123],
  [12000, 13000, 1327],
  [10004, 13000, 377],
  [13000, 14000, 4237],
  [10002, 14000, 66],
  [14000, 15000, 1714],
  [10001, 15000, 66],
  [15000, 110000, 4455]],
 [[20021, 20019, 1673],
  [20019, 20018, 896],
  [20018, 20014, 1154],
  [20014, 21000, 2023],
  [20020, 20016, 727],
  [20016, 20017, 723],
  [20017, 20015, 754],
  [20015, 21000, 312],
  [21000, 22000, 536],
  [20013, 22000, 66],
  [22000, 23000, 1039],
  [20011, 23000, 66],
  [23000, 24000, 4007],
  [20003, 24000, 66],
  [24000, 25000, 576],
  [25000, 110000, 6929]],
 [[30034, 30030, 1276],
  [30030, 30027, 784],
  [30027, 30025, 851],
  [30025, 30023, 771],
  [30023, 31000, 399],
  [30029, 30028, 496],
  [30028, 30026, 906],
  [30026, 30024, 576],
  [30024, 31000, 1207],
  [31000, 32000, 2178],
  [30022, 32000, 66],
  [32000, 110000

In [13]:
# BusPairs = []
# for (k,val) in enumerate (allFdrLen[0]):
#     BusPairs[]
    

In [14]:
df_col = df.iloc[0:n_rows,[1, 2, 4, 23, 24, 27, 25, 26, 27, 33, 36]]
#B,C,D,X,Y,AB,Y,Z,AB,AH, AK (TBD)
mpt_labels = df_col.iloc[:,10].unique()
#--------------------------------------------------------------------------
mpt_array = []
for (i,data) in enumerate(allFdrLen): #i indicates feeders 1,2,3 of allFdrLen
    fd_array = [x[0] for x in allFdrLen[i]]
    df_fd = df_col.loc[df['From'].isin(fd_array)] # get the column "From"
    n_gens = max(df_fd.iloc[:,9])
    mpt_labels_temp = df_fd.iloc[:,10].unique()
    mpt_array.append([mpt_labels_temp[0], n_gens])
print(mpt_array)
#-----------------------------------------------------------------------------------
mpt_labels_dict = {}
for (kk, label) in enumerate(mpt_labels):
    n_gens_total = 0
    for (k,val) in enumerate(mpt_array):
        if val[0] == label:
            n_gens_total += val[1]
    mpt_labels_dict[label] = n_gens_total
print(mpt_labels_dict)
    
#    sum((df_fd.iloc[:,23])*(df_fd.iloc[:,33]**2))/(n_gens**2)

[[0.03278439663835329, 10], [0.04578321834387734, 11], [0.0349189399921025, 10], [0.03921539264003361, 10]]
{0.03278439663835329: 10, 0.09873631308111741: 0, 0.04359394310926276: 0, 0.9659629653928166: 0, 1.5241734183386684: 0, 0.9188114500273051: 0, 1.0738137734381852: 0, 0.14019571283663096: 0, 1.0649377074144089: 0, 0.010316959543121191: 0, 5.1728092430679276: 0, 0.0018061520685570258: 0, 2.6484017231092634: 0, 8.49837192362949: 0, 0.04578321834387734: 11, 0.09807953051073307: 0, 0.28422265733383745: 0, 0.8857807599250577: 0, 0.01989503869455997: 0, 0.07914229973131696: 0, 0.18570527177618143: 0, 0.13661077463994958: 0, 0.6543841761350978: 0, 1.6054197143001891: 0, 7.643765723453057: 0, 0.5744764223395086: 0, 15.993516287583114: 0, 0.0349189399921025: 10, 0.08581958919689141: 0, 0.20959573777391305: 0, 0.33758624117756775: 0, 0.272975255816005: 0, 0.013573506454610376: 0, 0.09917416812804032: 0, 0.14186503520302457: 0, 0.5284910416359588: 0, 3.365355281757278: 0, 30.319444075009454:

In [ ]:
#calculation of equivalent collector system
Vrms_collection = df_col.iloc[0,2]
Vrms_MV = Vrms_collection
colPIdata = []
colPI_dict = {}
for (label, val) in mpt_labels_dict.items():
    #print(label)
    #print(val)
    n_gens = val
    df_col_temp = df_col[df_col.iloc[:,10] == label]
    #print(df_col)
    #print('_'*100)
    print(df_col_temp)
    
    R1_temp = sum(df_col_temp.iloc[:,3]*(df_col_temp.iloc[:,9]**2))/(n_gens**2)
    R1_temp = float("{:.10f}".format(R1_temp))
    ####### didn't get this equation of R #######
    
    X1_temp = sum(df_col_temp.iloc[:,4]*(df_col_temp.iloc[:,9]**2))/(n_gens**2)
    X1_temp = float("{:.10f}".format(X1_temp))
    
    B1_temp = sum(df_col_temp.iloc[:,5])
    B1_temp = float("{:.10f}".format(B1_temp))
    
    R0_temp = sum(df_col_temp.iloc[:,6]*(df_col_temp.iloc[:,9]**2))/(n_gens**2)
    R0_temp = float("{:.10f}".format(R0_temp))
    
    X0_temp = sum(df_col_temp.iloc[:,7]*(df_col_temp.iloc[:,9]**2))/(n_gens**2)
    X0_temp = float("{:.10f}".format(X0_temp))
    
    B0_temp = sum(df_col_temp.iloc[:,8])
    B0_temp = float("{:.10f}".format(B0_temp))

    
    line = [label, label, Vrms_collection, R1_temp, X1_temp, B1_temp, R0_temp, X0_temp, B0_temp, 0] # '0' is for the cable length concept
    print(line)
    #eqvPIdata[i] = P2P.usr_LineData(line, eqvPIdata[i])
    colPItemp = P2P.usr_LineData(line, PIattr.copy(), PIpara.copy()) # .copy() to avoid mutual editable of the dictionaries
    colPIdata.append(colPItemp)
    colPI_dict[label] = colPItemp

[0.03278439663835329, 0.03278439663835329, 34.5, 5.00326e-05, 4.37058e-05, 0.000339146, 0.000327844, 0.0001669796, 0.000339146, 0]


ZeroDivisionError: float division by zero

In [ ]:
mpt_labels_dict

### Read data for overhead line

In [ ]:
xldataframe = pd.ExcelFile(file_name)
sheetname = '3 OH line impedance'
df = xldataframe.parse(sheetname)

#transimission line parameters

R1t = df.iloc[0, 29]
R1t = float("{:.10f}".format(R1t))
X1t = df.iloc[0, 30]
X1t = float("{:.10f}".format(X1t))
B1t = df.iloc[0, 35] 
B1t = float("{:.10f}".format(B1t))

R0t = df.iloc[0, 31]
R0t = float("{:.10f}".format(R0t))
X0t = df.iloc[0, 32]
X0t = float("{:.10f}".format(X0t))
B0t = df.iloc[0, 36] 
B0t = float("{:.10f}".format(B0t))

### Update data for transformer

In [ ]:
# Read the transformer data (initialize variables)
sheets = xldataframe.sheet_names
#print(sheets)

# Find the index of the sheet that has XFMR data
XFMR_idx = [i for (i, item) in enumerate(sheets) if re.search('XFMR', item)]
df = xldataframe.parse(sheets[XFMR_idx[0]])
#df = xldataframe.parse(sheets[1])

# Find the sheet of collection system using Regex in Python
col_sys_idx = [i for (i, item) in enumerate(sheets) if re.search(r"\bcollection\b.*\bimpedance\b|\bimpedance\b.*\bcollection\b", item)]
#find sheet names that contain the words 'collection' and 'impedance,' but not necessarily in that order
#print(col_sys_idx)
sheets[col_sys_idx[0]]

xfmr_type = []
#xfmr_dicts = {}
i = 3
cnt = 3
while not pd.isnull(df.iloc[2,i]):
    print('tl',df.iloc[2,i])
    xfmr_type.append((i, df.iloc[2,i]))
    i += 1
#print(xfmr_type)
xfmr_3w = []
xfmr_2wdg = []
xfmr_3wdg = []
for (k, val) in enumerate(xfmr_type):
    if 'TER' in val[1].upper():
        xfmr_3w.append((val[0], val[1]))

for (k, val) in enumerate(xfmr_3w):
    if k%2 == 1:
        idx = xfmr_type.index(val)
        temp = xfmr_type[(idx - 5):(idx + 1)]
        xfmr_3wdg.append(temp)
        
set_3wdg = set() 
for (k, val) in enumerate(xfmr_3wdg):
    set_3wdg.update(set(val))

set_3wdg = sorted(set_3wdg)
set_2wdg = sorted(set(xfmr_type) - set(set_3wdg))

for (k, val) in enumerate(set_2wdg):
    if k%2 == 1:
        xfmr_2wdg.append(set_2wdg[k-1:k+1])
#print(xfmr_2wdg)

In [ ]:
#reading
MPTs = []
W3attr = {}
W3Trans = {}
for (k, val) in enumerate(xfmr_3wdg):
    i = val[0][0]
    #print(i)
    #while (df.iloc[3,i]) > 0:
    print("moving along", i)
    WVs = [(df.iloc[3,i]), (df.iloc[3,i+1]), (df.iloc[3,i+3])]
    #print(WVs)
    Rs = [(df.iloc[10,i]), (df.iloc[10,i+2]), (df.iloc[10,i+4])] #Might change to just the first one
    #print(Rs)
    Xs = [(df.iloc[11,i]), (df.iloc[11,i+2]), (df.iloc[11,i+4])]
    #print(Xs)
    MVA = [(df.iloc[12,i])]
    #print(MVA)
    line = [WVs, Rs, Xs, MVA]
    MPTdata = P2P.usr_MPTmkr(line, W3attr.copy(), W3Trans.copy())
    MPTs.append(MPTdata)
    
W2attr = {}
W2Trans = {}
for (k, val) in enumerate(xfmr_2wdg):
    i = val[0][0]
    WVs = [(df.iloc[3,i]), (df.iloc[3,i+1])]
    Rs = [(df.iloc[10,i])] #Might change to just the first one
    Xs = [(df.iloc[11,i])]
    MVA = [(df.iloc[12,i])]
    #print(MVA)
    line = [WVs, Rs, Xs, MVA]
    MPTdata = P2P.usr_MPTmkr_TwoWt_excel(line, W2attr.copy(), W2Trans.copy())
    MPTs.append(MPTdata)
print(MPTdata)

In [ ]:
# Add more xfmr if data in the collection system sheet and xfmr sheet dosen't match
if len(MPTs) < len(mpt_labels_dict):
    for i in range(0, (len(mpt_labels_dict) - len(MPTs))):
        MPTs.append(MPTs[-1])

Vrms_HV = df.iloc[3,3]
Vrms_LV = df.iloc[3, 2]

In [ ]:
#read GSU data
GSUres = df.iloc[10, 1]
GSUres = float("{:.10f}".format(GSUres))
GSUrect = df.iloc[11, 1]
GSUrect = float("{:.10f}".format(GSUrect))
GSUmva = df.iloc[12, 1]

GSUHV = df.iloc[3, 1]

GSULV = df.iloc[3, 2]

GSUfreq = Freq

In [ ]:
GSUres

### read data xfmr

## Choosing type of study

In [ ]:
for i in range(0, (len(mpt_labels_dict) - len(MPTs))):
    print('tine')

In [ ]:
all_feeder_len = []
for (kk,feeder) in enumerate(allFdrLen):
    PImpt, MPTbus, genbus, juncbus, endbus, longest_fdr, longest_bus, longest_len, longest_weight = ss.usr_get_feeder_info(feeder)
#     print(longest_fdr)
#     print(genbus)
    all_feeder_len.append(len(longest_fdr))
print(all_feeder_len)
n_bus_max = max(all_feeder_len)
# print(n_bus_max)

In [ ]:
#get project data from an excel file
prj_infor_files = utils.get_all_file_names(project_infor_dir, ".xlsx")

In [ ]:
#get project data from an excel file
prj_infor_files = utils.get_all_file_names(project_infor_dir, ".xlsx")
prj_infor_name = project_infor_dir + "\\" + prj_infor_files[0] + '.xlsx'
#wb2 = xlrd.open_workbook(prj_infor_name)

xldataframe = pd.ExcelFile(prj_infor_name)
prj_infor_sheets = xldataframe.sheet_names

df2 = xldataframe.parse(prj_infor_sheets[0])

#study_flag = int(input())
#1 for Harmonic study and 2 for TOV study
study_type = df2.iloc[17, 1].lower()
study_type.lower()
if re.search("^harmonic", study_type): 
    study_flag = 1
if re.search("^tov", study_type): 
    study_flag = 2
else:
    print("other study type has been chosen")
    
#Voltage source parameters for strong grid
R1s = df2.iloc[6, 1]
L1s = df2.iloc[7, 1]
R0s = df2.iloc[8, 1]
L0s = df2.iloc[9, 1]

#Voltage source parameters for weak grid
R1w = df2.iloc[12, 1]
L1w = df2.iloc[13, 1]
R0w = df2.iloc[14, 1]
L0w = df2.iloc[15, 1]

#email_infor
generate_email_flag = df2.iloc[19, 1].lower()
email_address = df2.iloc[20, 1]

#gen Type
project_type = df2.iloc[1,2].lower()

##TOV/TRV arrester data
if study_flag == 2: 
    df2 = xldataframe.parse(prj_infor_sheets[1])
    Varr_hv = df2.iloc[0,2]
    Varr_mpthv = df2.iloc[1,2]
    Varr_mptmv = df2.iloc[2,2]
    Varr_mv = df2.iloc[3,2]
    Varr_elbow = df2.iloc[4,2]

In [ ]:
PSCADlibs_dir = working_dir + "\\" + "PSCADlibs"
all_lib_files = utils.get_all_file_names(PSCADlibs_dir, ".pslx")

# 2. Start PSCAD

In [ ]:
versions = mhi.pscad.versions()

# Skip any 'Alpha' versions, if other choices exist
vers = [(ver, x64) for ver, x64 in versions if ver != 'Alpha']
if len(vers) > 0:
    versions = vers

# Skip any 'Beta' versions, if other choices exist
vers = [(ver, x64) for ver, x64 in versions if ver != 'Beta']
if len(vers) > 0:
    versions = vers
# Skip any 32-bit versions, if other choices exist
vers = [(ver, x64) for ver, x64 in versions if x64]
if len(vers) > 0:
    versions = vers


# Of any remaining versions, choose the "lexically largest" one.
version, x64 = sorted(versions)[-1]


In [ ]:
# Get all installed FORTRAN compiler versions
fortrans = mhi.pscad.fortran_versions()

# Skip 'GFortran' compilers, if other choices exist
vers = [ver for ver in fortrans if 'GFortran' not in ver]
if len(vers) > 0:
    fortrans = vers
    
# Order the remaining compilers, choose the last one (highest revision)
#fortran = sorted(fortrans)[-1]

# Launch PSCAD
#settings = {'fortran_version': fortran}
#pscad = mhi.pscad.launch(minimize=True, version=version, x64=x64, settings=settings)
# Launch PSCAD
if len(fortrans) > 0:
    fortran = sorted(fortrans)[-1]
    settings = {'fortran_version': fortran}
    pscad = mhi.pscad.launch(minimize=True, version=version, x64=x64, settings=settings)
else: 
    pscad = mhi.pscad.launch(minimize=True, version=version, x64=x64)
    
    fortran = ''

In [ ]:
# Select the corresponding extension

if (re.match('Intel 12',fortran)):
    fortran_ext = '.if12'
    print('Fortran 12 32-bit')

elif (re.match('Intel 15',fortran)):
    if(re.search(r'([0-9]+)-([\w.-]+)', fortran)):
        fortran_ext = '.if15'
        print('Fortran 15 64-bit')
    else:
        fortran_ext = '.if15_x86'
        print('Fortran 15 32-bit')

# Fortran 17 has same extension as Fortran 15            
elif (re.match('Intel 17',fortran)):
    if(re.search(r'([0-9]+)-([\w.-]+)', fortran)):
        fortran_ext = '.if15'
        print('Fortran 17 64-bit')
    else:
        fortran_ext = '.if15_x86'
        print('Fortran 17 32-bit')
        
elif (re.match('Intel 19',fortran)):
    if(re.search(r'([0-9]+)-([\w.-]+)', fortran)):
        fortran_ext = '.if18'
        print('Fortran 19 64-bit')
    else:
        fortran_ext = '.if18_x86'
        print('Fortran 19 32-bit')
    
else:
    fortran_ext = '.gf46'
    print('GFortran')
    

In [ ]:
fortran_ext = '.if18_x86'

In [ ]:
PSCAD_file_name = raw_files[0]
file_len = 10 # PSCAD file name is limitted by 10 characters
if(len(raw_files[0]) > file_len):
    PSCAD_file_name = raw_files[0][0:file_len]

#Setup up PSCAD
project_name = 'MainProject'
folder = os.getcwd()
pscad.new_workspace(project_name, working_dir + "\\" + "PSCADprj")

### load library

In [ ]:
#load all library in PSCADlibs
for (kk,ext_lib) in enumerate(all_lib_files):
    pscad.load(working_dir + "\\" + "PSCADlibs" + "\\" + ext_lib + ".pslx")

In [ ]:
pscad.create_case(PSCAD_file_name, working_dir + "\\" + "PSCADprj") 
project = pscad.project(PSCAD_file_name)
main = project.canvas('Main')
time.sleep(1)  

project.create_layer("Disable")
project.set_layer_state("Disable", "Disabled")
#
project.create_layer("PIfdr")
project.set_layer_state("PIfdr", "Enabled")
#
project.create_layer("eqvPI")
project.set_layer_state("eqvPI", "Disabled")
#
project.create_layer("Breaker")
project.set_layer_state("Breaker", "Enabled")
#
project.create_layer("Arrester")
project.set_layer_state("Arrester", "Enabled")
#
project.create_layer("Multimeter")
project.set_layer_state("Multimeter", "Enabled")
#

project.parameters(time_duration = 20, time_step = 100, sample_step = 300)
sim_set = "sim"
pscad.create_simulation_set(sim_set)
simset = pscad.simulation_set(sim_set)
#simset.add_tasks(PSCAD_file_name)

In [ ]:
main.parameters(size = '100X100')
#frozenset({'100X100', '11X17', '17X22', '22X34', '34X44', '85X11'})

In [ ]:
main.parameters(show_grid = 1)

In [ ]:
#Save simuilation output
project.parameters(PlotType="1", output_filename="$(Namespace).out")


In [ ]:
MPTs

### Generate collection system

In [ ]:
dictLabel = {}
dictOutChannel = {}
buslv_dict = {}
IIDcolPI = {}
resitor = 1e7

### Add LV bus and multimeter

In [ ]:
x_busLV = 50
multiLV_dict = {}
for (i,(k,val)) in enumerate(colPI_dict.items()):
    IID_list_bus = []
    IID_list_col = []
    IID_list_multi = []
#    print(i)
    p = val
    bus = main.create_bus((x_busLV, (i + 1)*20 - 2), (x_busLV, (i + 1)*20 + 2))
    name = "busLV" + str(i+1)
    bus.parameters(Name = name, BaseKV = Vrms_collection)
    bus.parameters(Name = name, BaseKV = Vrms_collection)
    IID_list_bus.append(bus.iid)
    buslv_dict[k] = IID_list_bus
    x_busLV, y_busLV = bus.get_location()
    
    mult_obj = main.add_component('master','multimeter', x_busLV + 6, y_busLV + 2)
    mult_obj.parameters(Name = 'mult_LV' + str(i+1))
    mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_LV', P = 'PLV_' + str(i+1), Q = 'QLV_' + str(i+1), Vrms = 'VLV_' + str(i+1))
    x_mult, y_mult = mult_obj.get_location()
    IID_list_multi.append(mult_obj.iid)
    multiLV_dict[k] = IID_list_multi
    main.create_wire((x_busLV, y_mult), (x_mult - 1, y_mult))
    
    CMP = main.add_component('master', 'resistor', x_mult - 2 , y_mult + 2) 
    CMP.parameters(R = resitor)
    CMP.rotate_right()
    ground = main.add_component('master', 'ground', x_mult -2  ,  y_mult + 4) 
    ground.rotate_right()
    #creat wire
    main.create_wire((x_mult - 2 , y_mult + 2), (x_mult - 2, y_mult))


## Add GSU transformer

In [ ]:
GSU_dict = {}

for (k,val) in multiLV_dict.items():
    print(k)
    GSU_list =[]
    obj = main.component(val[0])
    multi_LV= obj.get_location()
    for (kk,ext_lib) in enumerate(all_lib_files):
        try:
            GENxfmr = main.add_component(ext_lib, 'Electranix_Scale_1', 5, 4)
            TOV_lib = ext_lib
            GENxfmr.parameters(V1 = GSUHV)  
            break
        except:
            GENxfmr.delete()
    GENxfmr.delete() 
    GSU_trans = main.add_component(ext_lib, 'xfmr_2w_scaled', multi_LV[0] + 10, multi_LV[1])
    GSU_trans.parameters(Name="GSU_trans_" + str(k), Tmva=GSUmva, f=GSUfreq, YD1="1", YD2="0", Lead="2", Xl=GSUrect, Ideal="0", NLL="0.0 [pu]", CuL="0.0 [pu]", 
                         Tap="0", View="1", Dtls="0", V1=GSUHV, V2=GSULV, Enab="0", Sat="1", Xair="0.2 [pu]", Tdc="1.0 [s]", Xknee="1.25 [pu]", Txk="0.1 [s]", Im1="0.4 [%]")
    GSU_trans.mirror()
    GSU_list.append(GSU_trans.iid)
    GSU_dict[k] = GSU_list
    GSU_trans.add_to_layer("TOVTRV")
    GSU_transloc = GSU_trans.get_location()
    main.create_wire((GSU_transloc[0] - 2, GSU_transloc[1]), (multi_LV[0] + 1, multi_LV[1]))
    
    CMP = main.add_component('master', 'resistor', GSU_transloc[0] , GSU_transloc[1] + 4) 
    CMP.parameters(R = resitor)
    CMP.rotate_right()
    ground = main.add_component('master', 'ground', GSU_transloc[0] , GSU_transloc[1] + 6) 
    ground.rotate_right()
    #creat wire
    main.create_wire((GSU_transloc[0] , GSU_transloc[1] + 2), (GSU_transloc[0] , GSU_transloc[1] + 4))


In [ ]:
GSU_list

### Add numbers genorator on GSU transformer

In [ ]:
for (label, val) in mpt_labels_dict.items():
    for (i, gsu_loc) in GSU_dict.items():
        if i == label:
            obj = main.component(gsu_loc[0])
            gsuloc= obj.get_location()
            tap_obj = main.add_component('master', 'const',  gsuloc[0] - 1, gsuloc[1] - 2)
            tap_obj.parameters(Value = val)

### Add Mutimeter MV in the left PI

In [ ]:
multimv_dict = {}
for (i, (k,val)) in enumerate(GSU_dict.items()):
    multimv_dict_list = []
    obj = main.component(val[0])
    gsuloc= obj.get_location()
    # Add multimeter
    mult_obj = main.add_component('master','multimeter',gsuloc[0] + 8, gsuloc[1])
    mult_obj.parameters(Name = 'mult_xfmrMV' + str(i + 1) )
    mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_MV', P = 'PMV' + str(i +1), Q = 'QMV' + str(i +1), Vrms = 'VMV'+ str(i +1))
    mult_obj.add_to_layer("Multimeter")
    multimv_dict_list = mult_obj.iid
    multimv_dict[k] = multimv_dict_list
    main.create_wire((gsuloc[0] + 4, gsuloc[1]), (gsuloc[0] + 7, gsuloc[1]))
    

### Add equivalent PI section

In [ ]:
for (k,val) in multimv_dict.items():
    print(val)
    IID_list_col = []
    p = colPI_dict[k]
    obj = main.component(val)
    GSU_transloc= obj.get_location()
    colPIcmp = main.add_component(p[0]['Lib'], p[0]['CType'], GSU_transloc[0] + 10, GSU_transloc[1])
    colPIcmp.parameters(**p[1])
#    print(**p[1])
    colPIcmp.add_to_layer("colPI")
    x_colPI, y_colPI = colPIcmp.get_location()
    IID_list_col.append(colPIcmp.iid)
    IIDcolPI[k] = IID_list_col
    
    main.create_wire((GSU_transloc[0] + 1, GSU_transloc[1]), (x_colPI - 2, y_colPI))

### Add MPTs

In [ ]:
# Add MPTs
condinate_MPT = {}
MPT_dict = {}
multi_dict = {}

for (i, (k, val) ) in enumerate(IIDcolPI.items()):
    t = MPTs[i]
    temp = MPTs[i][0]['CType']
    multi_list = []
    # Add transformer 
    Numb = i + 1 
    mpt_name = 'T' + str(Numb)
    obj = main.component(val[0])
    PI_loc= obj.get_location()
    Xt =  PI_loc[0] + 30
    Yt =  PI_loc[1] + 0
    MPTcmp = main.add_component(t[0]['Lib'], t[0]['CType'], Xt, Yt)
    MPTcmp.parameters(**t[1])
    MPTcmp.parameters(Name = mpt_name)
    Xmpt, Ympt = MPTcmp.get_location()
#   Comdinate_list.append(MPTcmp)
    condinate_MPT[Numb] = Ympt
    MPT_dict.update({mpt_name:MPTcmp.iid})
    if temp == 'xfmr-3p3w2':
        label_obj = main.add_component('master', 'datalabel', Xmpt - 3,  Ympt - 1)
        label_name = 'Tap' + str(Numb)
        label_obj.parameters(Name = label_name)

        # Add ground and resistor for transfomer
        main.add_component('master', 'ground', Xmpt + 2, Ympt + 2)

        CMP = main.add_component('master', 'resistor', Xmpt + 4, Ympt) 
        CMP.parameters(R = 1e6) 


        main.add_component('master', 'ground', Xmpt + 8, Ympt) 
        #creat wire
        main.create_wire((Xmpt + 3, Ympt), (Xmpt + 4, Ympt))
        main.create_wire((Xmpt + 6, Ympt), (Xmpt + 8, Ympt))
        main.create_wire((Xmpt - 1,Ympt + 2),(Xmpt, Ympt + 2), (Xmpt, Ympt + 2), (Xmpt + 2, Ympt + 2))

        # Add multimeter
        mult_obj = main.add_component('master','multimeter',Xmpt + 16, Ympt)
        mult_obj.parameters(Name = 'mult_xfmrHV_' + str(Numb) )
        mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_HV', P = 'PHV_' + str(Numb), Q = 'QHV_' + str(Numb), Vrms = 'VHV_'+ str(Numb))
        mult_obj.add_to_layer("Multimeter")
        multi_list = mult_obj.iid
        multi_dict[k] = multi_list
        x_mult, y_mult = mult_obj.get_location()
        main.create_wire((Xmpt, Ympt - 4),(x_mult - 4, y_mult - 4), (x_mult -1, y_mult)) # creat wire for multimeter 1

        mult_obj = main.add_component('master','multimeter',Xmpt - 10, Ympt)
        mult_obj.parameters(Name = 'mult_xfmrMV_' + str(Numb))
        mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_MV', P = 'PMV_' + str(Numb), Q = 'QMV_' + str(Numb), Vrms = 'VMV_' + str(Numb), VolI="Vmv"+str(Numb) , CurI="Imv" + str(Numb))
        mult_obj.add_to_layer("Multimeter")
        x_mult, y_mult = mult_obj.get_location()
        main.create_wire((Xmpt - 3, Ympt), (Xmpt - 9, Ympt)) # creat wire for multimeter 2

        main.create_wire((PI_loc[0] + 4, PI_loc[1]), (x_mult - 1, y_mult)) # creat wire for multimeter
    if temp == 'xfmr-3p2w':
        label_obj = main.add_component('master', 'datalabel', Xmpt - 3,  Ympt - 1)
        label_name = 'Tap' + str(Numb)
        label_obj.parameters(Name = label_name)

        # Add ground and resistor for transfomer
        ground = main.add_component('master', 'ground', Xmpt - 1, Ympt + 4)
        ground.rotate_right()

        CMP = main.add_component('master', 'resistor', Xmpt - 1, Ympt +2) 
        CMP.parameters(R = 1e6) 
        CMP.rotate_right()

#         main.add_component('master', 'ground', Xmpt + 8, Ympt) 
#         #creat wire
#         main.create_wire((Xmpt + 3, Ympt), (Xmpt + 4, Ympt))
#         main.create_wire((Xmpt + 6, Ympt), (Xmpt + 8, Ympt))
#         main.create_wire((Xmpt - 1,Ympt + 2),(Xmpt, Ympt + 2), (Xmpt, Ympt + 2), (Xmpt + 2, Ympt + 2))

        # Add multimeter
        mult_obj = main.add_component('master','multimeter',Xmpt + 16, Ympt)
        mult_obj.parameters(Name = 'mult_xfmrHV_' + str(Numb) )
        mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_HV', P = 'PHV_' + str(Numb), Q = 'QHV_' + str(Numb), Vrms = 'VHV_'+ str(Numb))
        mult_obj.add_to_layer("Multimeter")
        multi_list = mult_obj.iid
        multi_dict[k] = multi_list
        x_mult, y_mult = mult_obj.get_location()
        main.create_wire((Xmpt + 2, Ympt), (x_mult -1, y_mult)) # creat wire for multimeter 1

        mult_obj = main.add_component('master','multimeter',Xmpt - 10, Ympt)
        mult_obj.parameters(Name = 'mult_xfmrMV_' + str(Numb))
        mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_MV', P = 'PMV_' + str(Numb), Q = 'QMV_' + str(Numb), Vrms = 'VMV_' + str(Numb), VolI="Vmv"+str(Numb) , CurI="Imv" + str(Numb))
        mult_obj.add_to_layer("Multimeter")
        x_mult, y_mult = mult_obj.get_location()
        main.create_wire((Xmpt - 3, Ympt), (Xmpt - 9, Ympt)) # creat wire for multimeter 2

        main.create_wire((PI_loc[0] + 4, PI_loc[1]), (x_mult - 1, y_mult)) # creat wire for multimeter
#         print('tien')
    
    # Add OLTC
    oltc = main.add_component('ERCOTLib', 'OLTC', Xmpt, Ympt + 8)
    oltc.parameters(label_name, pos_step="16", neg_step="16", pos_reg="10 [%]", neg_reg="-10 [%]", v_set="1.0 [pu]", v_band="5 [%]", td="30 [s]", init_step="0", enab_t="1 [s]", lock="0", )

    label_obj = main.add_component('master', 'datalabel', Xmpt - 3, Ympt + 8)
    label_name = 'VMV_' + str(Numb)

    label_obj.parameters(Name = label_name)
    
    label_obj = main.add_component('master', 'datalabel', Xmpt + 3, Ympt + 8)
    label_name = 'Tap' + str(Numb)
    label_obj.parameters(Name = label_name)
    
    tap_obj = main.add_component('master', 'const',  Xmpt - 3, Ympt + 12)
    tap_obj.parameters(Value = 0)
    
    label_obj = main.add_component('master', 'datalabel', Xmpt + 2, Ympt + 12)
    label_name = 'InitTap' + str(Numb)
    label_obj.parameters(Name = label_name)
    main.create_wire((Xmpt - 1, Ympt + 12), (Xmpt + 2, Ympt + 12))

    

### Add bus POI

In [ ]:
Ymult_loc = []
Vrms_POI = Vrms_HV
for (k, val) in multi_dict.items():
    obj = main.component(val)
    Xmult , Ymult = obj.get_location()
    Ymult_loc.append(Ymult)
    main.create_wire((Xmult + 1, Ymult), (Xmult + 6, Ymult))
bus = main.create_bus((Xmult + 6, min(Ymult_loc) - 4), (Xmult + 6, max(Ymult_loc) +4))
name = "busPOI"
bus.parameters(Name = name, BaseKV = Vrms_collection)
x_busMV, y_busMV = bus.get_location()


const_obj = main.add_component('master', 'const',  Xmult - 4, min(Ymult_loc) - 10)
const_obj.parameters(Value = Vrms_LV)

label_obj = main.add_component('master', 'datalabel', Xmult, min(Ymult_loc) - 10)
label_name = 'Vrms_LV'
label_obj.parameters(Name = label_name)
main.create_wire((Xmult - 2, min(Ymult_loc) - 10), (Xmult, min(Ymult_loc) - 10))


const_obj = main.add_component('master', 'const',  Xmult + 4, min(Ymult_loc) - 10)
const_obj.parameters(Value = Vrms_MV)

label_obj = main.add_component('master', 'datalabel', Xmult + 8, min(Ymult_loc) - 10)
label_name = 'Vrms_MV'
label_obj.parameters(Name = label_name)
main.create_wire((Xmult + 6, min(Ymult_loc) - 10), (Xmult + 8, min(Ymult_loc) - 10))

const_obj = main.add_component('master', 'const',  Xmult + 12, min(Ymult_loc) - 10)
const_obj.parameters(Value = Vrms_HV)

label_obj = main.add_component('master', 'datalabel', Xmult + 16, min(Ymult_loc) - 10)
label_name = 'Vrms_HV'
label_obj.parameters(Name = label_name)
main.create_wire((Xmult + 14, min(Ymult_loc) - 10), (Xmult + 16, min(Ymult_loc) - 10))


const_obj = main.add_component('master', 'const',  Xmult + 20, min(Ymult_loc) - 10)
const_obj.parameters(Value = Vrms_POI)

label_obj = main.add_component('master', 'datalabel', Xmult + 24, min(Ymult_loc) - 10)
label_name = 'Vrms_POI'
label_obj.parameters(Name = label_name)
main.create_wire((Xmult + 22, min(Ymult_loc) - 10), (Xmult + 24, min(Ymult_loc) - 10))



### ADD transimision line and multimeter for POI

In [ ]:
trans_line = main.add_component('master', 'newpi', x_busMV + 6, y_busMV + 4)
trans_line.parameters(Name = 'Transimission_Line', PU = 3, Estim = 0, View = 2, F = Freq, len = 1, VR2 = Vrms_HV, 
                           RPUP2 = R1t, XLPUP2 = X1t, BPUP2 = B1t, RPUZ2 = R0t, XLPUZ2 = X0t, BPUZ2 = B0t)
Xt_line, Yt_line= trans_line.get_location()

# Add multimeter
mult_obj = main.add_component('master','multimeter',Xt_line + 30, Yt_line)
mult_obj.parameters(Name = 'mult_POI')
mult_obj.parameters(MeasP = 1, MeasQ = 1 , RMS = 1, BaseV = 'Vrms_POI', P = 'Ppoi', Q = 'Qpoi', Vrms = 'Vpoi')
x_mult, y_mult = mult_obj.get_location()

main.create_wire((Xt_line + 4, Yt_line), (x_mult - 1, y_mult))

main.create_wire((x_busMV, Yt_line), (Xt_line -2 , Yt_line))
# add data labels for POI
label_obj = main.add_component('master', 'datalabel', x_mult, y_mult + 4)
label_name = 'Ppoi'
label_obj.parameters(Name = label_name)
dictLabel.update({label_name:label_obj.iid})

label_obj = main.add_component('master', 'datalabel', x_mult, y_mult + 6)
label_name = 'Qpoi'
label_obj.parameters(Name = label_name)
dictLabel.update({label_name:label_obj.iid})

label_obj = main.add_component('master', 'datalabel', x_mult, y_mult + 8)
label_name = 'Vpoi'
label_obj.parameters(Name = label_name)
dictLabel.update({label_name:label_obj.iid})

all_label_name = ['Ppoi', 'Qpoi', 'Vpoi']
for label_name in all_label_name:
    label, out_iid = ss.usr_add_an_output_channel(main, label_name, dictLabel)
    dictOutChannel.update({label:out_iid})
    
# Add grid
vrtgrid = main.add_component('ERCOTLib', 'VRTGrid', x_mult + 14, y_mult)
vrtgrid.parameters(VLL=str(Vrms_POI), vrtOnOff="0", frtOnOff="0", )
vrtgrid.mirror()
x_grid, y_grid = vrtgrid.get_location()
main.create_wire((x_mult + 1, y_mult), (x_grid - 8, y_grid))

# Add graphs for POI
dictGraphs = {}
all_label_name = ['Ppoi', 'Qpoi', 'Vpoi']
loc = (x_mult, y_mult + 10)
name, iid = ss.usr_add_plots(main, loc, all_label_name, dictOutChannel)
dictGraphs.update({name:iid})

### Add data label

In [ ]:
# add data labels for HV
distance = 8
for (i, (k, val) ) in enumerate(IIDcolPI.items()):
    # add data labels for LV
    label_obj = main.add_component('master', 'datalabel', Xt_line -2, Yt_line + 4 + (i*distance))
    label_name = 'PLV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})

    label_obj = main.add_component('master', 'datalabel', Xt_line -2, Yt_line + 6 + (i*distance))
    label_name = 'QLV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})
    
    label_obj = main.add_component('master', 'datalabel', Xt_line -2, Yt_line + 8 + (i*distance))
    label_name = 'VLV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})
    
    
    # add data labels for MV_
    label_obj = main.add_component('master', 'datalabel', Xt_line + 8 , Yt_line + 4 + (i*distance))
    label_name = 'PMV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})

    label_obj = main.add_component('master', 'datalabel', Xt_line + 8, Yt_line + 6 + (i*distance))
    label_name = 'QMV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})
    
    label_obj = main.add_component('master', 'datalabel', Xt_line + 8, Yt_line + 8 + (i*distance))
    label_name = 'VMV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})
    
    # add data labels for HV
    label_obj = main.add_component('master', 'datalabel', Xt_line + 18, Yt_line + 4 + (i*distance))
    label_name = 'PHV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})

    label_obj = main.add_component('master', 'datalabel', Xt_line + 18, Yt_line + 6 + (i*distance))
    label_name = 'QHV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})
    
    label_obj = main.add_component('master', 'datalabel', Xt_line + 18, Yt_line + 8 + (i*distance))
    label_name = 'VHV_' + str(i+1)
    label_obj.parameters(Name = label_name)
    dictLabel.update({label_name:label_obj.iid})

    #add Output Channel for P,Q,V LV
    all_label_name = ['PLV_' + str(i+1), 'QLV_' + str(i+1), 'VLV_' + str(i+1)]
    for label_name in all_label_name:
        label, out_iid = ss.usr_add_an_output_channel(main, label_name, dictLabel)
        dictOutChannel.update({label:out_iid})

    #add Output Channel for P,Q,V MV_
    all_label_name = ['PMV_' + str(i+1), 'QMV_' + str(i+1), 'VMV_' + str(i+1)]
    for label_name in all_label_name:
        label, out_iid = ss.usr_add_an_output_channel(main, label_name, dictLabel)
        dictOutChannel.update({label:out_iid})
    
    #add Output Channel for P,Q,V HV
    all_label_name = ['PHV_' + str(i+1), 'QHV_' + str(i+1), 'VHV_' + str(i+1)]
    for label_name in all_label_name:
        label, out_iid = ss.usr_add_an_output_channel(main, label_name, dictLabel)
        dictOutChannel.update({label:out_iid})
        

### Add graphs

In [ ]:
x_temp = 0
y_temp = 0 
bus_temp = 0
for (i, (k, val)) in enumerate(buslv_dict.items()): 
    obj = main.component(val[0])
    x_obj, y_obj = obj.get_location()
    if y_temp <= y_obj:
        y_temp = y_obj
        x_temp = x_obj
        bus_temp = val[0]
# Add graphs for LV, MV, HV   
x_dist = 34
y_dist = 60
for (i, (k, val)) in enumerate(IIDcolPI.items()):
    all_label_name = ['PLV_' + str(i+1), 'QLV_' + str(i+1), 'VLV_' + str(i+1)]
    loc = (x_temp + 0 * x_dist, y_temp + (i + 1) * y_dist)
    name, iid = ss.usr_add_plots(main, loc, all_label_name, dictOutChannel)
    dictGraphs.update({name:iid})
    
    all_label_name = ['PMV_' + str(i+1), 'QMV_' + str(i+1), 'VMV_' + str(i+1)]
    loc = (x_temp + 1 * x_dist , y_temp + (i + 1) * y_dist)
    name, iid = ss.usr_add_plots(main, loc, all_label_name, dictOutChannel)
    dictGraphs.update({name:iid})
    
    all_label_name = ['PHV_' + str(i+1), 'QHV_' + str(i+1), 'VHV_' + str(i+1)]
    loc = (x_temp + 2 * x_dist , y_temp + (i + 1) * y_dist)
    name, iid = ss.usr_add_plots(main, loc, all_label_name, dictOutChannel)
    dictGraphs.update({name:iid})

In [ ]:
project.build()
project.save()
pscad.save_workspace(project_name, working_dir + "\\" + "PSCADprj")


In [ ]:
time_stop = time.time()
print("--- %s seconds ---" % (time_stop - time_start))

# Send notification email

In [ ]:
#if generate_email_flag == 'yes':
#now = datetime.datetime.now()
    #to = email_address
    #computer_name = os.environ["COMPUTERNAME"] + " computer"
    #
    #subject = 'Your PSCAD plant model is done. Great work!'
    #body = 'Just want to let you know the PSCAD plant model is done at ' + now.strftime("%H:%M %m-%d-%Y")  + \
    #'\n The project files are located in ' + working_dir + ' on ' + computer_name +\
    #'\n Way to do!'
    #mailutils.send_gmail(sender, password, to, subject, body, attachments=None)
    #mailutils.send_outlook_mail(to, subject, body, attachments=None)

## Finish equivalent test 